#### 获取所有2板及以上的票

In [3]:
beg_date = '20220101'
end_date = '20221231'
date2pool = dict()

In [4]:
from lh.stockinfo import AllStock_daily, SingleStock_daily
from lh.time import Trade_time
from tqdm import tqdm

tt = Trade_time()
asd = AllStock_daily()
ssd = SingleStock_daily()

In [5]:
trade_dates = tt.getTradeDateList()
for trade_date in tqdm(trade_dates):
    pool = asd.get_least_2bans_list(trade_date=trade_date)
    if len(pool):
        date2pool[trade_date] = pool

100%|██████████| 243/243 [06:55<00:00,  1.71s/it]


In [6]:
len(date2pool)

243

#### 2板及以上票的连扳概率

In [7]:
for date in date2pool.keys():
    pool = date2pool[date]
    date2pool[date] = {ts_code: dict() for ts_code in pool}

In [8]:
for date in date2pool.keys():
    for ts_code in date2pool[date]:
        nextdate = tt.nextTradeDate(date=date)
        date2pool[date][ts_code]['nextIs_ban'] = nextdate in date2pool and ts_code in date2pool[nextdate]       

In [9]:
num_2banplus = 0
num_2banplus_still = 0
for date in date2pool:
    num_2banplus += len(date2pool[date])
    for ts_code in date2pool[date]:
        if date2pool[date][ts_code]['nextIs_ban']:
            num_2banplus_still += 1
print(num_2banplus_still,num_2banplus,num_2banplus_still/num_2banplus)

2129 4799 0.4436340904355074


#### 查看无脑2进3成功率

In [10]:
date2pool_ban_2_only = dict()

In [13]:
date2pool_ban_2_only.clear()
for trade_date in tqdm(trade_dates):
    pool = asd.get_only_k_bans_list(2,trade_date,False,True)
    if len(pool)>0:
        date2pool_ban_2_only[trade_date] = pool

100%|██████████| 243/243 [02:02<00:00,  1.99it/s]


In [14]:
date2pool_ban_3_only = dict()
for trade_date in tqdm(trade_dates):
    pool = asd.get_only_k_bans_list(3,trade_date,False,True)
    if len(pool)>0:
        date2pool_ban_3_only[trade_date] = pool

100%|██████████| 243/243 [04:00<00:00,  1.01it/s]


In [15]:
num_ban_2_only = 0
num_ban_2_only_stillban = 0
for trade_date in tqdm(trade_dates[:-1]):
    num_ban_2_only += len(date2pool_ban_2_only[trade_date])
    next_date = tt.nextTradeDate(trade_date)
    if next_date in date2pool_ban_3_only:
        num_ban_2_only_stillban += len(set(date2pool_ban_2_only[trade_date])&set(date2pool_ban_3_only[next_date]))
print(num_ban_2_only_stillban,num_ban_2_only,num_ban_2_only_stillban/num_ban_2_only)

100%|██████████| 242/242 [00:00<00:00, 691.98it/s]

968 2567 0.37709388391118037
